# If you want to access the version you have already modified, click "Edit"
# If you want to access the original sample code, click "...", then click "Copy & Edit Notebook"
# 增加mixup

In [1]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
        # print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
_exp_name = "sample"

In [3]:
# Import necessary packages.
import numpy as np
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset

# This is for the progress bar.
from tqdm.auto import tqdm
import random

In [4]:
myseed = 6666  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

import torchvision
print(torchvision.__version__)

0.21.0+cu124


## **Transforms**
Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Please refer to PyTorch official website for details about different transforms.

In [5]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.RandomResizedCrop(128, antialias=True),
    # You may add some transforms here.
    
    # 2. 几何变换 (增加数据多样性)
    transforms.RandomHorizontalFlip(p=0.5),        # 水平翻转
    transforms.TrivialAugmentWide(),
    
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
])


## **Datasets**
The data is labelled by the name, so we load images and label while calling '__getitem__'

加入mixup 将两张图片混合

In [6]:
class FoodDataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None, mode = 'train', mixup = False, alpha = 0.4):
        super(FoodDataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files is not None: #为k折提供的接口
            self.files = files
        print(f"One {path} sample",self.files[0])
        self.transform = tfm
        self.alpha = alpha
        self.mixup = mixup
        self.mode = mode
        self.num_classes = 11 # Food-11 有 11 类
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        #im = self.data[idx]
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        
        # mixup部分
        if self.mixup and self.mode == 'train':
            idx2 = np.random.randint(len(self.files))
            fname2 = self.files[idx2]
            im2 = Image.open(fname2)
            im2 = self.transform(im2)
            # train一定有label
            label2 = int(fname2.split("/")[-1].split("_")[0])
            # 生成混合比例 lambda (从 Beta 分布采样)
            lam = np.random.beta(self.alpha, self.alpha)
            # 3. 混合图片 (线性组合) [cite: 76]
            # im 和 im2 都是 Tensor，可以直接加减
            mixed_im = lam * im + (1 - lam) * im2

            # 4. 混合标签 (生成概率向量) [cite: 77]
            # 初始化全 0 向量
            label_vec = torch.zeros(self.num_classes)
            # label 1 的概率设为 lam
            label_vec[label] = lam
            # label 2 的概率累加 (1-lam) -> 用累加是为了处理 label==label2 的情况
            label_vec[label2] += (1 - lam)
            return mixed_im, label_vec

        
        return im,label


## 新的交叉熵（torch的交叉熵不适用于向量，仅适用于3这种明确类别）  

In [7]:
import torch.nn as nn
import torch.nn.functional as F

class MixupCrossEntropyLoss(nn.Module):
    def __init__(self):
        super(MixupCrossEntropyLoss, self).__init__()

    def forward(self, preds, targets):
        # preds: 模型的输出 logits (Batch_Size, 11)
        # targets: Mixup 生成的混合标签 (Batch_Size, 11)
        
        # 1. 对模型输出做 Log Softmax (数值上比 log(softmax) 更稳定)
        log_probs = F.log_softmax(preds, dim=1)
        
        # 2. 计算公式: - sum( target * log_prob )
        # dim=1 表示在类别维度求和
        loss = -torch.sum(targets * log_probs, dim=1) #逐元素相乘
        
        # 3. 对 Batch 取平均
        return loss.mean()

In [8]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 128, 128]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 128, 128]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 64, 64]

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]
            
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
        )
        self.fc = nn.Sequential(
            nn.Linear(512*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 11)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

# [Step 2] 使用 ResNet18 模型
import torchvision.models as models

def get_model():
    # 加载 ResNet18 结构，不使用预训练权重
    model = models.resnet18(pretrained=False)
    
    # 修改全连接层 (fc)，因为 ResNet18 默认输出 1000 类，我们需要 11 类
    # model.fc.in_features 是 ResNet18 最后一层的输入维度 (通常是 512)
    model.fc = nn.Linear(model.fc.in_features, 11) 
    return model

# 验证一下模型是否能跑
# print(get_model())

In [9]:
_dataset_dir = "../input/ml2022spring-hw3b/food11"

In [10]:
from sklearn.model_selection import KFold
import glob
import numpy as np

# 1. 收集所有数据路径
train_dir = os.path.join(_dataset_dir, "training")
val_dir = os.path.join(_dataset_dir, "validation")

all_train_files = sorted([os.path.join(train_dir, x) for x in os.listdir(train_dir) if x.endswith(".jpg")])
all_val_files = sorted([os.path.join(val_dir, x) for x in os.listdir(val_dir) if x.endswith(".jpg")])
all_files = np.array(all_train_files + all_val_files) 

# ==========================================
# 🔧 技巧 1：极小数据集调试模式开关
# ==========================================
debug_mode = False  # 想要正式跑的时候，改成 False

device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 64
seed = 6666

if debug_mode:
    print("⚠️ 警告：当前处于调试模式！")
    # [技巧 1] 只取前 100 张图，确保几秒钟能加载完
    all_files = all_files[:100] 
    n_epochs = 2      # 只跑 2 个 Epoch 看有没有报错
    n_folds = 2       # KFold 至少需要 2 折，但我们下面只会跑第 1 折
    patience = 999    # 调试时不需要早停
else:
    # 正式模式
    n_epochs = 80     
    n_folds = 5       
    patience = 15     

# 2. 开始 K-Fold 循环
kf = KFold(n_splits=n_folds, shuffle=True, random_state=seed)

# enumerate 会返回 (0, ...), (1, ...)
for fold_idx, (train_idx, val_idx) in enumerate(kf.split(all_files)):
    
    print(f"\n----- Starting Fold {fold_idx + 1} / {n_folds} -----")
    
    # 划分当前 Fold 的文件
    fold_train_files = all_files[train_idx]
    fold_val_files = all_files[val_idx]
    
    # 构建 Dataset (注意: train 开启 Mixup)
    train_set = FoodDataset(path=train_dir, tfm=train_tfm, files=fold_train_files, mixup=True, mode='train')
    valid_set = FoodDataset(path=val_dir, tfm=test_tfm, files=fold_val_files, mixup=False, mode='valid')
    
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    
    # 初始化模型
    model = get_model().to(device)
    
    prev_ckpt_path = f"/kaggle/input/hw3-version5-model/sample_fold{fold_idx+1}_best.ckpt"
    if os.path.exists(prev_ckpt_path):
        print(f"Loading fold {fold_idx+1} from {prev_ckpt_path}")
        # model.load_state_dict(torch.load(prev_ckpt_path))
        model.load_state_dict(torch.load(prev_ckpt_path, map_location=device))
    else:
        print(f"Fold {fold_idx+1} checkpoint not found, training from scratch.")
    # 定义 Loss
    criterion_mixup = MixupCrossEntropyLoss()
    criterion_normal = nn.CrossEntropyLoss()
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)
    
    best_acc = 0
    stale = 0
    
    # 3. 训练循环
    for epoch in range(n_epochs):
        # --- Training ---
        model.train()
        train_loss = []
        train_accs = []
        
        # 使用 enumerate 方便获取 batch 索引 i
        for i, batch in enumerate(tqdm(train_loader, desc=f"Fold {fold_idx+1} Epoch {epoch+1}")):
            imgs, labels = batch
            imgs, labels = imgs.to(device), labels.to(device)
            
            logits = model(imgs)
            
            # 计算 Loss (Mixup)
            loss = criterion_mixup(logits, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # 计算 Accuracy (注意 Mixup 标签处理！)
            # labels 是 (B, 11) 的概率向量，logits 是 (B, 11)
            # 我们比较两个向量中最大值的位置是否一致
            acc = (logits.argmax(dim=-1) == labels.argmax(dim=-1)).float().mean()
            
            train_loss.append(loss.item())
            train_accs.append(acc.item())
            
            # ==========================================
            # 🔧 技巧 5：快速中断循环 (Train)
            # ==========================================
            if debug_mode and i >= 5: # 跑完 5 个 batch 就强制跳出
                print("调试模式：Training 强制中断...")
                break

        avg_train_loss = sum(train_loss) / len(train_loss)
        avg_train_acc = sum(train_accs) / len(train_accs)
        
        # --- Validation ---
        model.eval()
        valid_loss = []
        valid_accs = []
        
        with torch.no_grad():
            for i, batch in enumerate(valid_loader):
                imgs, labels = batch
                imgs, labels = imgs.to(device), labels.to(device)
                
                logits = model(imgs)
                
                # 验证集没有 Mixup，labels 是普通的 (B,) 索引
                loss = criterion_normal(logits, labels)
                acc = (logits.argmax(dim=-1) == labels).float().mean()
                
                valid_loss.append(loss.item())
                valid_accs.append(acc.item())
                
                # ==========================================
                # 🔧 技巧 5：快速中断循环 (Valid)
                # ==========================================
                if debug_mode and i >= 5:
                    print("调试模式：Validation 强制中断...")
                    break
                
        avg_valid_loss = sum(valid_loss) / len(valid_loss)
        avg_valid_acc = sum(valid_accs) / len(valid_accs)
        
        print(f"[Fold {fold_idx+1} | {epoch+1}/{n_epochs}] Train Acc: {avg_train_acc:.4f}, Valid Acc: {avg_valid_acc:.4f}")
        
        # 保存模型
        if avg_valid_acc > best_acc:
            print(f"New Best Fold {fold_idx+1} Model! Acc: {avg_valid_acc:.4f}")
            torch.save(model.state_dict(), f"{_exp_name}_fold{fold_idx+1}_best.ckpt")
            best_acc = avg_valid_acc
            stale = 0
        else:
            stale += 1
            if stale > patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
    
    # ==========================================
    # 🔧 技巧 1补充：只跑第一个 Fold 就结束
    # ==========================================
    if debug_mode:
        print("调试模式：只运行第一个 Fold，程序结束。")
        break


----- Starting Fold 1 / 5 -----
One ../input/ml2022spring-hw3b/food11/training sample ../input/ml2022spring-hw3b/food11/training/0_0.jpg
One ../input/ml2022spring-hw3b/food11/validation sample ../input/ml2022spring-hw3b/food11/training/0_1.jpg


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Loading fold 1 from /kaggle/input/hw3-version5-model/sample_fold1_best.ckpt


Fold 1 Epoch 1:   0%|          | 0/167 [00:00<?, ?it/s]

[Fold 1 | 1/80] Train Acc: 0.5450, Valid Acc: 0.7093
New Best Fold 1 Model! Acc: 0.7093


...

[Fold 5 | 22/80] Train Acc: 0.6350, Valid Acc: 0.7307


Fold 5 Epoch 23:   0%|          | 0/167 [00:00<?, ?it/s]

[Fold 5 | 23/80] Train Acc: 0.6327, Valid Acc: 0.7199


Fold 5 Epoch 24:   0%|          | 0/167 [00:00<?, ?it/s]

[Fold 5 | 24/80] Train Acc: 0.6436, Valid Acc: 0.7353
Early stopping at epoch 24


## Testing and generate prediction CSV

In [11]:
# [Step 4] 极简版：集成预测 (Ensemble)
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

# 准备测试集
test_set = FoodDataset(os.path.join(_dataset_dir, "test"), tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

# 初始化全零数组 [样本数, 11个类别]
final_logits = np.zeros((len(test_set), 11))

# 遍历每一个 Fold 的模型
for fold_idx in range(n_folds):
    model_path = f"{_exp_name}_fold{fold_idx+1}_best.ckpt"
    
    if not os.path.exists(model_path):
        print(f"Model {model_path} not found, skipping.")
        continue
        
    print(f"Inference with {model_path}...")
    
    # 加载模型
    model = get_model().to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    fold_preds = []
    with torch.no_grad():
        for data, _ in tqdm(test_loader):
            logits = model(data.to(device))
            fold_preds.append(logits.cpu().numpy())
    
    # 直接累加 Logits
    final_logits += np.concatenate(fold_preds)

# 直接对“和”取最大值索引 (不需要除以模型数量)
prediction = np.argmax(final_logits, axis=1)


One ../input/ml2022spring-hw3b/food11/test sample ../input/ml2022spring-hw3b/food11/test/0001.jpg
Inference with sample_fold1_best.ckpt...


  0%|          | 0/53 [00:00<?, ?it/s]

Inference with sample_fold2_best.ckpt...


  0%|          | 0/53 [00:00<?, ?it/s]

Inference with sample_fold3_best.ckpt...


  0%|          | 0/53 [00:00<?, ?it/s]

Inference with sample_fold4_best.ckpt...


  0%|          | 0/53 [00:00<?, ?it/s]

Inference with sample_fold5_best.ckpt...


  0%|          | 0/53 [00:00<?, ?it/s]

In [12]:
#create test csv
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1,len(test_set)+1)]
df["Category"] = prediction
df.to_csv("submission.csv",index = False)
print("Ensemble submission saved!")

Ensemble submission saved!
